In [2]:
import numpy as np
import pandas as pd
import rebound
import spock
import matplotlib.pyplot as plt
% matplotlib inline
np.random.seed(2)

/Users/alysa/miniconda3/envs/astroconda/lib/python3.5/site-packages/xgboost/__init__.py:29: FutureWarning: Python 3.5 support is deprecated; XGBoost will require Python 3.6+ in the near future. Consider upgrading to Python 3.6+.
  FutureWarning)


In [3]:
df = pd.read_csv("../generate-tables/cumulative_koi_gaia_dr2_mass_inc.csv", comment="#")

In [4]:
def sys_to_rebound_sim(sysid):
    
    m_earth = 3e-6
    year = 2 * np.pi # one year in REBOUND time (in units where G=1)
    day = year / 365 # one day in REBOUND time (in units where G=1)
    
    system = df[df['kepid'] == sysid].sort_values(by='koi_period')
    
    sim = rebound.Simulation()
        
    sim.add(m=system['koi_smass'].iloc[0])

    for i, planet in system.iterrows():
        mass = planet['koi_pmass'] * m_earth
        period = planet['koi_period'] * day
        ecc = 0
        inc = planet['koi_pinc']
        pomega = np.random.uniform(0, 2*np.pi)
        Omega = np.random.uniform(0, 2*np.pi)
        f = np.random.uniform(0, 2*np.pi)
    
        sim.add(m=mass, P=period, e=ecc, inc=inc, pomega=pomega, Omega=Omega,f=f)
            
    sim.move_to_com()
    
    return sim
        

In [5]:
def insert_test_particle(sim, sysid, dirname):
    
    m_earth = 3e-6
    mass_test = 3.671 * m_earth
    ecc_test = 0
    inc_test = np.random.rayleigh(np.random.rayleigh(1)) * np.pi / 180
    pomega_test = np.random.uniform(0, 2*np.pi)
    Omega_test = np.random.uniform(0, 2*np.pi)
    f_test = np.random.uniform(0, 2*np.pi)
    
    N_planets = len(sim.particles[1:])
    N_pair = N_planets - 1
    
    m_star = sim.particles[0].m
    
    test_particle_systems = []
    
    for i in range(N_pair):
        new_sim = sim.copy()
        
        a_inner = new_sim.particles[i+1].a
        a_outer = new_sim.particles[i+2].a
        
        mass_inner = new_sim.particles[i+1].m
        mass_outer = new_sim.particles[i+2].m
        
        hill_inner = a_inner * (mass_inner / m_star / 3) ** (1/3)
        hill_outer = a_outer * (mass_outer / m_star / 3) ** (1/3)
        
        spacing = (a_outer - a_inner) / (hill_inner + hill_outer)
        a_test = a_inner + spacing * hill_inner
        
        new_sim.add(m=mass_test, a=a_test, e=ecc_test, inc=inc_test, pomega=pomega_test, Omega=Omega_test, f=f_test)
        ps = new_sim.particles
        
        maxd = np.array([p.d for p in new_sim.particles[1:new_sim.N_real]]).max()
        new_sim.exit_max_distance = 100*maxd

        for p in new_sim.particles[1:new_sim.N_real]:
            rH = p.a*(p.m/3./new_sim.particles[0].m)**(1./3.)
            p.r = rH
        
        new_sim.move_to_com()
        new_sim.collision="line"
        new_sim.integrator="whfast"
        new_sim.dt = 0.05*ps[1].P
        new_sim.ri_whfast.safe_mode=0
        
        archive_file_name = dirname + "/system-" + str(sysid) + "-" + str(i) + ".bin"
        
        new_sim.save(archive_file_name)
        
    return None
        

In [6]:
for sysid in df['kepid'].unique():
    insert_test_particle(sys_to_rebound_sim(sysid), sysid, "simulation-archives/original-archives")

In [ ]:
# count number of systems as f'n of multiplicity

n_2_planet = 0
n_3_planet = 0
n_4_planet = 0
n_5plus_planet = 0

for sysid in df[df['koi_prad'] < 20]['kepid'].unique():
    if len(df[df['kepid'] == sysid]) == 2:
        n_2_planet += 1
    elif len(df[df['kepid'] == sysid]) == 3:
        n_3_planet +=1
    elif len(df[df['kepid'] == sysid]) == 4:
        n_4_planet +=1
    elif len(df[df['kepid'] == sysid]) >= 5:
        n_5plus_planet +=1

In [ ]:
print(n_2_planet, n_3_planet, n_4_planet, n_5plus_planet)